In [10]:
!pip install xgboost

   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---- ----------------------------------- 7.6/72.0 MB 36.2 MB/s eta 0:00:02
   -------- ------------------------------- 14.4/72.0 MB 34.9 MB/s eta 0:00:02
   ------------ --------------------------- 22.3/72.0 MB 36.2 MB/s eta 0:00:02
   ----------------- ---------------------- 32.2/72.0 MB 38.6 MB/s eta 0:00:02
   ----------------------- ---------------- 41.4/72.0 MB 39.9 MB/s eta 0:00:01
   --------------------------- ------------ 49.5/72.0 MB 39.9 MB/s eta 0:00:01
   -------------------------------- ------- 57.7/72.0 MB 40.0 MB/s eta 0:00:01
   ------------------------------------ --- 65.5/72.0 MB 39.8 MB/s eta 0:00:01
   ---------------------------------------  71.8/72.0 MB 39.5 MB/s eta 0:00:01
   ---------------------------------------- 72.0/72.0 MB 36.7 MB/s  0:00:02


In [98]:
import pandas as pd
import seaborn as sns
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score,classification_report
from xgboost import XGBClassifier

In [52]:
df = sns.load_dataset('titanic')

In [53]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    int64   
dtypes: bool(1), category(2), float64(2), int64(5), object(5)
memory usage: 86.7+ KB


In [54]:
df['alone'] = df['alone'].astype(int)


In [75]:
features=['sex','pclass', 'age','fare','embarked','alone','survived']
df2 = df[features].copy()


In [76]:
df2.head()

,sex,pclass,age,fare,embarked,alone,survived
0,male,3,22.0,7.2500,S,0,0
1,female,1,38.0,71.2833,C,0,1
2,female,3,26.0,7.9250,S,1,1
3,female,1,35.0,53.1000,S,0,1
4,male,3,35.0,8.0500,S,1,0


In [77]:
df2['sex'] = df2['sex'].map({'male':0, 'female':1})

In [78]:
df2['embarked'] = df2['embarked'].fillna('S') ## only 2 missing so should be fine


In [79]:
df2['embarked'] = df2['embarked'].map({'C': 0,'Q': 1,'S': 2})


In [80]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   sex       891 non-null    int64  
 1   pclass    891 non-null    int64  
 2   age       714 non-null    float64
 3   fare      891 non-null    float64
 4   embarked  891 non-null    int64  
 5   alone     891 non-null    int64  
 6   survived  891 non-null    int64  
dtypes: float64(2), int64(5)
memory usage: 48.9 KB


In [81]:
df2.corr()

,sex,pclass,age,fare,embarked,alone,survived
sex,1.000000,-0.131900,-0.093254,0.182333,-0.108262,-0.303646,0.543351
pclass,-0.131900,1.000000,-0.369226,-0.549500,0.162098,0.135207,-0.338481
age,-0.093254,-0.369226,1.000000,0.096067,-0.030394,0.198270,-0.077221
fare,0.182333,-0.549500,0.096067,1.000000,-0.224719,-0.271832,0.257307
embarked,-0.108262,0.162098,-0.030394,-0.224719,1.000000,0.063532,-0.167675
alone,-0.303646,0.135207,0.198270,-0.271832,0.063532,1.000000,-0.203367
survived,0.543351,-0.338481,-0.077221,0.257307,-0.167675,-0.203367,1.000000


In [88]:
X = df2.drop('survived', axis=1)
y = df2['survived']


In [89]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [90]:
train_median_age = X_train['age'].median()

In [91]:
X_train['age'] = X_train['age'].fillna(train_median_age)
X_test['age'] = X_test['age'].fillna(train_median_age)

In [92]:
xgb = XGBClassifier(eval_metric='logloss')
param_grid = {
    'n_estimators': [10, 25, 50, 100],     # Number of trees
    'max_depth': [2, 3, 5, 7],             # Depth of the forest
    'learning_rate': [0.01, 0.1, 0.2, 0.25] # Step size
}


grid_search = GridSearchCV(param_grid=param_grid, estimator=xgb, cv=3, verbose=1)
grid_search.fit(X, y)

Fitting 3 folds for each of 64 candidates, totalling 192 fits


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.","XGBClassifier...ree=None, ...)"
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'learning_rate': [0.01, 0.1, ...], 'max_depth': [2, 3, ...], 'n_estimators': [10, 25, ...]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",None
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",3
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;- >

In [93]:
best_model = grid_search.best_estimator_
print(f"Best Parameters Found: {grid_search.best_params_}")

Best Parameters Found: {'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 25}


In [100]:
prediction = grid_search.predict(X_test)
accuracy = accuracy_score(y_test, prediction)
print(f"Final Test Accuracy: {accuracy*100:.2f}%")


Final Test Accuracy: 90.67%


In [103]:
print(classification_report(y_test,prediction))

              precision    recall  f1-score   support

           0       0.88      0.98      0.93       168
           1       0.96      0.78      0.86       100

    accuracy                           0.91       268
   macro avg       0.92      0.88      0.90       268
weighted avg       0.91      0.91      0.90       268



In [96]:
model_filename = 'titanic_xgb_v1.pkl'
joblib.dump(best_model, model_filename)
print(f"Model saved to {model_filename}")

Model saved to titanic_xgb_v1.pkl
